# Crawling Drama
 - 드라마 목록 위키([링크](https://ko.wikipedia.org/wiki/2018%EB%85%84_%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%ED%85%94%EB%A0%88%EB%B9%84%EC%A0%84_%EB%93%9C%EB%9D%BC%EB%A7%88_%EB%AA%A9%EB%A1%9D))
<!-- -인스타 / 페이스북 대체 페이지([링크](https://pikdo.net/u/styleandstar/2064045967)) -->

In [1]:
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip

     | 481kB 9.3MB/s
  Running setup.py bdist_wheel for tqdm ... - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-uid6jdgr/wheels/41/18/ee/d5dd158441b27965855b1bbae03fa2d8a91fe645c01b419896
Successfully built tqdm
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
pymc3 3.6 has requirement joblib<0.13.0, but you'll have joblib 0.13.0 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [2]:
import pandas as pd
import numpy as np
import sys
import requests
import json
from bs4 import BeautifulSoup as bs
import json
from datetime import *
from urllib.request import Request, urlopen
from tqdm import tqdm_notebook as tqdm
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Get Data from Wikipedia

In [0]:
url = 'https://ko.wikipedia.org/wiki/2018%EB%85%84_%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%ED%85%94%EB%A0%88%EB%B9%84%EC%A0%84_%EB%93%9C%EB%9D%BC%EB%A7%88_%EB%AA%A9%EB%A1%9D'
res = requests.get(url).text
soup = bs(res, 'lxml')

### Extract data from table on Wikipedia
- data contains **title, link, date **...

In [0]:
def makelist(table):
  url_base = 'https://ko.wikipedia.org'
  result = []
  allrows = table.findAll('tr')
  for row in allrows:
    result.append([])
    allcols = row.findAll('td')
    for col in allcols:
      texts = col.text.replace('《','').replace('》','').replace('\n','')
      links = col.findAll('a')
      if links == []:
         result[-1].append(texts +";")
      else:
        for link in links:
          result[-1].append(texts +";"+ url_base + link['href'])
  return result

def getcol(table):
  result = []
  allrows = table.findAll('th')
  for row in allrows:
    result.append(row.text.replace('\n',''))
  return result


### Make Dataframe
- drop useless data from dataframe 

In [0]:
tables = soup.find_all('table',{'class':'wikitable'})[1:4]
len(tables)
dra_list = []
for table in tables:
  dra_list.extend(makelist(table)[1:])
df_dra = pd.DataFrame(dra_list)

df_dra.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,조선미인별전;https://ko.wikipedia.org/wiki/%EC%A1%B0...,경민선;,김대현;,2018.01.062018.01.07;,2;,KBS1;,특집;,;,예;,;,;,None
1,문집;,신하은;,이윤정;https://ko.wikipedia.org/wiki/%EC%9D%B4%EC...,2018.01.06;,1;,tvN;,단막(토);,;,예;,;,;,None
2,낫 플레이드(Not Played);,강민수;,유종선;,2018.01.13;,1;,tvN;,단막(토);,;,예;,;,;,None
3,리턴;https://ko.wikipedia.org/wiki/%EB%A6%AC%ED%...,최경미;,주동민이정림;,2018.01.172018.03.22;,34 (17);,SBS;,수목;,;,예;,;,2회 연장;,None
4,우리 집은 맛나 된장 맛나;,윤조;,김상호;https://ko.wikipedia.org/wiki/%EA%B9%80%EC...,2018.01.20;,1;,tvN;,단막(토);,;,예;,;,;,None


In [0]:
df_dra.iloc[35,3] = '2018.04.282018.06.03;'
df_dra = df_dra[[0,3]]
df_dra['Title'] = ''
df_dra['Link'] = ''
df_dra['Start_date'] = ''
df_dra['End_date'] = ''
df_dra['Title'] = df_dra[0].map(lambda x:x.split(';')[0])
df_dra['Link'] = df_dra[0].map(lambda x:x.split(';')[-1])
df_dra['Start_date'] = df_dra[3].map(lambda x:x[:10])
df_dra['End_date'] = df_dra[3].map(lambda x:x[10:].replace(';',''))

df_dra = df_dra[['Title','Link','Start_date','End_date']]
df_dra.head()

,Title,Link,Start_date,End_date
0,조선미인별전,https://ko.wikipedia.org/wiki/%EC%A1%B0%EC%84%...,2018.01.06,2018.01.07
1,문집,,2018.01.06,
2,낫 플레이드(Not Played),,2018.01.13,
3,리턴,https://ko.wikipedia.org/wiki/%EB%A6%AC%ED%84%...,2018.01.17,2018.03.22
4,우리 집은 맛나 된장 맛나,,2018.01.20,


### 2018년 2~3분기 방영한 드라마
-  2018년 4월 1일부터 2018년 9월 30일 사이에 방영한 드라마

In [0]:
df_dra = df_dra[df_dra['End_date'] >= '2018.04.01']
df_dra = df_dra[df_dra['Link'] != '']

## Get Ratings Data from Wikipedia

### make list with links

In [0]:
li_dra = df_dra['Link'].tolist()

### Extract data from table on Wikipedia
- 시청률 데이터 추출

In [0]:
def makelist2(table):
  url_base = 'https://ko.wikipedia.org'
  result = []
  allrows = table.findAll('tr')
  for row in allrows:
    result.append([])
    allcols = row.findAll('td')
    if len(allcols) > 2:
      for col in allcols[-2:]:
        texts = col.text.replace('《','').replace('》','').replace(' ','').replace('\n','').replace('%','')
        result[-1].append(texts)
    elif len(allcols) == 2:
      for col in allcols[-1:]:
        texts = col.text.replace('《','').replace('》','').replace(' ','').replace('\n','').replace('%','')
        result[-1].append(texts)
  return result


### make rating scores
- 드라마별 평균 시청률 구하기

In [0]:
score_list = []
for li in tqdm(li_dra):
  exceptions = ['https://ko.wikipedia.org/wiki/%EB%AF%B8%EC%8A%A4%ED%84%B0_%EC%85%98%EC%83%A4%EC%9D%B8',
                'https://ko.wikipedia.org/wiki/%EB%9D%BC%EC%9D%B4%ED%94%84_(2018%EB%85%84_%EB%93%9C%EB%9D%BC%EB%A7%88)',
                'https://ko.wikipedia.org/wiki/%EB%82%B4_%EB%92%A4%EC%97%90_%ED%85%8C%EB%A6%AC%EC%9A%B0%EC%8A%A4',
                'https://ko.wikipedia.org/wiki/KBS_%EB%93%9C%EB%9D%BC%EB%A7%88_%EC%8A%A4%ED%8E%98%EC%85%9C#2018년']
  res = requests.get(li).text
  soup = bs(res, 'lxml')
  tables = soup.find_all('table',{'class':'wikitable'})
  if li in exceptions[:-1]:
    scores = [np.nanmean([float(x) for x in li if x]) for li in makelist2(tables[0]) if li]
    score = round(np.nanmean(scores),2)
  elif li == exceptions[-1]:
    score = 0
  else :
    if 1 < len(tables) <= 2: 
      scores = [np.nanmean([float(x) for x in li if x]) for li in makelist2(tables[0]) if li]
      score = round(np.nanmean(scores),2)
    elif len(tables) > 2:
      scores = [np.nanmean([float(x) for x in li if x]) for li in makelist2(tables[1]) if li]
      score = round(np.nanmean(scores),2)
    else:
      score = 0
  score_list.append(score)
  

/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm_notebook.py:88: TqdmExperimentalWarning: Detect Google Colab 0.0.1a2 and thus load dummy ipywidgets package. Note that UI is different from that in Jupyter. See https://github.com/tqdm/tqdm/pull/640
  " See https://github.com/tqdm/tqdm/pull/640".format(colab.__version__), TqdmExperimentalWarning)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: Mean of empty slice


### Put rating score in dataframe
 - 데이터셋에 시청률 정보 추가

In [0]:
df_dra['Score'] = score_list
df_dra.head()

,Title,Link,Start_date,End_date,Score
12,으라차차 와이키키,https://ko.wikipedia.org/wiki/%EC%9C%BC%EB%9D%...,2018.02.05,2018.04.17,2.01
13,파도야 파도야,https://ko.wikipedia.org/wiki/%ED%8C%8C%EB%8F%...,2018.02.12,2018.08.31,0.00
15,키스 먼저 할까요,https://ko.wikipedia.org/wiki/%ED%82%A4%EC%8A%...,2018.02.20,2018.04.24,10.05
16,인형의 집,https://ko.wikipedia.org/wiki/%EC%9D%B8%ED%98%...,2018.02.26,2018.07.20,13.23
17,추리의 여왕 2,https://ko.wikipedia.org/wiki/%EC%B6%94%EB%A6%...,2018.02.28,2018.04.19,6.07


In [0]:
df_dra = df_dra[df_dra['Score'] > 0].sort_values(by = ['Score'],ascending=False)

In [0]:
df_dra.to_csv('gdrive/My Drive/공모전/Lpoint/데이터셋 만들기/drama.csv')

In [0]:
df_dra = pd.read_csv('gdrive/My Drive/공모전/Lpoint/데이터셋 만들기/drama.csv',index_col = 0)

### 월별 드라마 데이터 만들기

In [168]:
df_dra['Start_date'] = df_dra['Start_date'].map(lambda x : '2018.04.01' if x < '2018.04.01' else x)
df_dra['End_date'] = df_dra['End_date'].map(lambda x : '2018.09.30' if x > '2018.09.30' else x)
df_dra['months'] = (df_dra['End_date'].map(lambda x:int(x[5:7]))-df_dra['Start_date'].map(lambda x:int(x[5:7])))+1
df_dra.head()

,Title,Link,Start_date,End_date,Score,months
24,같이 살래요,https://ko.wikipedia.org/wiki/%EA%B0%99%EC%9D%...,2018.04.01,2018.09.09,28.16,6
40,내일도 맑음,https://ko.wikipedia.org/wiki/%EB%82%B4%EC%9D%...,2018.05.07,2018.09.30,16.08,5
58,미스터 션샤인,https://ko.wikipedia.org/wiki/%EB%AF%B8%EC%8A%...,2018.07.07,2018.09.30,14.13,3
16,인형의 집,https://ko.wikipedia.org/wiki/%EC%9D%B8%ED%98%...,2018.04.01,2018.07.20,13.23,4
65,끝까지 사랑,https://ko.wikipedia.org/wiki/%EB%81%9D%EA%B9%...,2018.07.23,2018.09.30,11.69,3


In [0]:
df = df_dra[['Title','Start_date','End_date','Score','months']]

In [170]:
#convert columns to datetime
df['Start_date'] = pd.to_datetime(df['Start_date'])
df['End_date'] = pd.to_datetime(df['End_date'])

#repeat rows
df = df.loc[df.index.repeat(df['months'])]
#group by index with transform for date ranges
df['Date'] =(df.groupby(level=0)['Start_date']
                         .transform(lambda x: pd.period_range(start=x.iat[0], periods=len(x), freq='M')))
df['Date'] = df['Date'].map(lambda x:str(x))
#unique default index
df_dra = df.reset_index(drop=True)[['Title','Date','Score']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## 드라마에 등장한 상품 가져오기
- *'네이버 TV 속 이 상품'* 서비스를 제공하는 팅글([링크](http://www.tingle.kr))에서 데이터 수집



In [0]:
keywords = df_dra[df_dra['Score'] > df_dra['Score'].quantile(0.5)]['Title'].tolist()

In [98]:
keywords[:10]

['같이 살래요',
 '같이 살래요',
 '같이 살래요',
 '같이 살래요',
 '같이 살래요',
 '같이 살래요',
 '같이 살래요',
 '같이 살래요',
 '같이 살래요',
 '같이 살래요']

### 드라마 id 가져오기


In [0]:
dra_id = []
for keyword in tqdm(keywords):
  url = 'https://api.infomore.co.kr/tingle/web/0.9/programs_search.json.html?page=1&_=T151503502&keyword={}&limit=200&token=null&category=1%7C2%7C3%7C7%7C8%7C9%7C10&genre=2'.format(keyword)
  res = requests.get(url)
  items = res.json().get('searchPrograms')
  try:
    dra_id.append({
        'title': keyword,
        'id':[str(item.get('id')) for item in items if item.get('title') == keyword]
    })
  except TypeError:
    pass

/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm_notebook.py:88: TqdmExperimentalWarning: Detect Google Colab 0.0.1a2 and thus load dummy ipywidgets package. Note that UI is different from that in Jupyter. See https://github.com/tqdm/tqdm/pull/640
  " See https://github.com/tqdm/tqdm/pull/640".format(colab.__version__), TqdmExperimentalWarning)


In [0]:
df_id = pd.DataFrame(dra_id)

### 드라마 id를 통해 상품 id 가져오기
- 드라마 마다 300개의 ppl 상품 추출

In [0]:
dra_items = []
for key in tqdm(dra_id):
  if key['id'] != []:
    url = 'https://api.infomore.co.kr/tingle/web/0.9/items_program.json.html?page=1&_=T153252785&id={}&limit=300&category=1%7C8&order=1&token=null&disable='.format(key['id'][0])
    res = requests.get(url)
    items = res.json().get('programItems')
    dra_items.append({
            'title': key['title'],
            'id':[str(item.get('id')) for item in items]})

### 상품 id를 활용해 상품 정보 가져오기

In [0]:
item_lists2 = []
for dra in tqdm(dra_items[:]):
  item_list = []
  for key in tqdm(dra['id']):
    url = 'https://api.infomore.co.kr/tingle/web/0.9/item.json.html?id={}&location=web'.format(key)
    res = requests.get(url)
    info = res.json()
    item_list.append([info.get('brandName'), info.get('itemTypeName')])
  item_lists2.append({
      'title': dra['title'],
      'lists': item_list
  })

### 명품만 남기기
 - 상품들 중 명품에 해당하는 상품만 남기기

In [0]:
lotte = pd.read_csv("gdrive/My Drive/공모전/Lpoint/데이터셋 만들기/lotte.csv")
keys = lotte[['PD_BRA_NM','CLAC3_NM']].apply(lambda x: '{}_{}'.format(x[0],x[1]), axis=1)

In [0]:
full_items = []
for li in tqdm(item_lists2):
  full_items.append([])
  full_items[-1].append(li['title'])
  full_items[-1].append(['_'.join(l).replace(' ','') for l in li['lists']])
  

### 브랜드 + ~~상품 분류~~ 별로 등장 수 count
 - 상품 분류를 구별해내기 어려움
  - 예) 가방(남성가방?/여성가방?), 셔츠(남성상의?/여성상의?)

In [0]:
drama_count = {}
for ti in tqdm(full_items):
  brands_count = {}
  for key in keys:
    brand = key.split('_')[0]
    
    if brand not in brands_count:
      brands_count[brand] = len([i for i in ti[1] if brand in i.split('_')[0]])
    else :
      brands_count[brand] += len([i for i in ti[1] if brand in i.split('_')[0]])
    drama_count[ti[0]] = brands_count


In [0]:
drama_count

In [0]:
df_bd = pd.DataFrame.from_dict(drama_count, orient='index').reset_index()
df_bd.rename({'index':'Title'},axis=1,inplace=True)
df_bd.head()

,Title,루이까또즈,엘르,탠디,SK-II,VDL,VT코스메틱,겔랑,끌레드뽀보떼,닥터자르트,...,조르쥬레쉬,페이,노이어,이로,로저비비에,아닉구딸,샤오미,워너원,사투르니아,헤르메스
0,같이 살래요,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,검법남녀,0,0,0,0,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,0
2,그녀로 말할 것 같으면,26,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,기름진 멜로,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,김비서가 왜 그럴까,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [0]:
df_bd.to_csv('gdrive/My Drive/공모전/Lpoint/데이터셋 만들기/in_drama.csv', index=False)

In [0]:
df_bd = pd.read_csv('gdrive/My Drive/공모전/Lpoint/데이터셋 만들기/in_drama.csv')

In [202]:
df_d = pd.merge(df_dra,df_bd, on=['Title'], how = 'inner')
# colnames = ['months','Start_date', 'End_date']
# df_d.drop(colnames, axis=1, inplace=True)
# df_d.set_index(['Date','Title'],inplace=True)
df_d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80 entries, 0 to 79
Columns: 389 entries, Title to 헤르메스
dtypes: float64(1), int64(386), object(2)
memory usage: 243.8+ KB


In [203]:
df_d.head()

,Title,Date,Score,루이까또즈,엘르,탠디,SK-II,VDL,VT코스메틱,겔랑,...,조르쥬레쉬,페이,노이어,이로,로저비비에,아닉구딸,샤오미,워너원,사투르니아,헤르메스
0,같이 살래요,2018-04,28.16,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,같이 살래요,2018-05,28.16,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,같이 살래요,2018-06,28.16,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,같이 살래요,2018-07,28.16,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,같이 살래요,2018-08,28.16,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [0]:
df_d.iloc[:,3:] = df_d.iloc[:,3:].apply(lambda x: x*df_d['Score'])

In [0]:
def normalize(x):
  return (x-np.mean(x))/(np.std(x))

In [206]:
df_d[df_d['Date']=='2018-04']

,Title,Date,Score,루이까또즈,엘르,탠디,SK-II,VDL,VT코스메틱,겔랑,...,조르쥬레쉬,페이,노이어,이로,로저비비에,아닉구딸,샤오미,워너원,사투르니아,헤르메스
0,같이 살래요,2018-04,28.16,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,28.16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,인형의 집,2018-04,13.23,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21,우리가 만난 기적,2018-04,11.03,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,22.06,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23,데릴남편 오작두,2018-04,10.55,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,슈츠,2018-04,8.96,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46,부잣집 아들,2018-04,7.97,207.22,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,47.82,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52,착한마녀전,2018-04,7.92,205.92,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72,스위치 - 세상을 바꿔라,2018-04,6.47,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,6.47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [207]:
df_d[df_d['Date']=='2018-04'].iloc[:,3:].apply(lambda x: sum(x), axis=0).reset_index().head()

,index,0
0,루이까또즈,413.14
1,엘르,0.00
2,탠디,0.00
3,SK-II,0.00
4,VDL,0.00


In [0]:
df_4 = df_d[df_d['Date']=='2018-04'].iloc[:,3:].apply(lambda x: sum(x), axis=0).reset_index()
df_4.columns = ['PD_BRA_NM','BRA_SC']
df_4['BRA_SC'] = normalize(df_4['BRA_SC'])
df_4['MONTH']='2018-04'

df_5 = df_d[df_d['Date']=='2018-05'].iloc[:,3:].apply(lambda x: sum(x), axis=0).reset_index()
df_5.columns = ['PD_BRA_NM','BRA_SC']
df_5['BRA_SC'] = normalize(df_5['BRA_SC'])
df_5['MONTH']='2018-05'

df_6 = df_d[df_d['Date']=='2018-06'].iloc[:,3:].apply(lambda x: sum(x), axis=0).reset_index()
df_6.columns = ['PD_BRA_NM','BRA_SC']
df_6['BRA_SC'] = normalize(df_6['BRA_SC'])
df_6['MONTH']='2018-06'

df_7 = df_d[df_d['Date']=='2018-07'].iloc[:,3:].apply(lambda x: sum(x), axis=0).reset_index()
df_7.columns = ['PD_BRA_NM','BRA_SC']
df_7['BRA_SC'] = normalize(df_7['BRA_SC'])
df_7['MONTH']='2018-07'

df_8 = df_d[df_d['Date']=='2018-08'].iloc[:,3:].apply(lambda x: sum(x), axis=0).reset_index()
df_8.columns = ['PD_BRA_NM','BRA_SC']
df_8['BRA_SC'] = normalize(df_8['BRA_SC'])
df_8['MONTH']='2018-08'

df_9 = df_d[df_d['Date']=='2018-09'].iloc[:,3:].apply(lambda x: sum(x), axis=0).reset_index()
df_9.columns = ['PD_BRA_NM','BRA_SC']
df_9['BRA_SC'] = normalize(df_9['BRA_SC'])
df_9['MONTH']='2018-09'

In [0]:
df_45 = pd.concat([df_4,df_5])
df_67 = pd.concat([df_6,df_7])
df_89 = pd.concat([df_8,df_9])
df_4567 = pd.concat([df_45,df_67])
df_t = pd.concat([df_4567,df_89])

In [0]:
df_bt = df_t.groupby(['MONTH','PD_BRA_NM'])["BRA_SC"].mean().reset_index()

### 상품분류 카운트

In [0]:
cate_count = {}
for ti in tqdm(full_items):
  cates_count = {}
  for key in keys:
    cate = key.split('_')[1]
    
    if cate not in cates_count:
      cates_count[cate] = len([i for i in ti[1] if i.split('_')[1] in cate])
    else :
      cates_count[cate] += len([i for i in ti[1] if i.split('_')[1] in cate])
    cate_count[ti[0]] = cates_count


In [0]:
cate_count

In [0]:
df_ct = pd.DataFrame.from_dict(cate_count, orient='index').reset_index()
df_ct.rename({'index':'Title'},axis=1,inplace=True)
df_ct.head()

,Title,2단우산,3단우산,BB/파운데이션/컴팩트류,공병/모델링팩전용도구,귀걸이,기름종이,기타남성화,기타모자,기타보석류,...,캐쥬얼크로스백,캐쥬얼힙색,크림/밤/오일,팔찌,패션액세서리세트,페이셜클렌저,페이셜팩류,펜던트,학생용가방,헤드웨어
0,같이 살래요,0,0,0,0,928,0,0,10,0,...,0,0,0,0,0,0,0,0,32,0
1,검법남녀,0,0,0,0,348,0,0,0,0,...,0,0,0,0,0,0,0,0,7,0
2,그녀로 말할 것 같으면,0,0,0,0,319,0,0,0,0,...,0,0,330,88,0,0,0,0,28,0
3,기름진 멜로,0,0,0,0,841,0,0,0,0,...,0,0,0,0,0,0,0,0,35,0
4,김비서가 왜 그럴까,0,0,0,0,1537,0,0,0,0,...,0,0,0,88,0,0,0,0,33,0


In [0]:
df_ct.to_csv('gdrive/My Drive/공모전/Lpoint/데이터셋 만들기/in_drama_c.csv', index=False)

In [0]:
df_cd = pd.read_csv('gdrive/My Drive/공모전/Lpoint/데이터셋 만들기/in_drama_c.csv')

In [254]:
df_c = pd.merge(df_dra,df_cd, on=['Title'], how = 'inner')
# colnames = ['Link', 'Start_date', 'End_date','Unnamed: 0']
# df_c.drop(colnames, axis=1, inplace=True)
df_c.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80 entries, 0 to 79
Columns: 177 entries, Title to 헤드웨어
dtypes: float64(1), int64(174), object(2)
memory usage: 111.2+ KB


In [255]:
df_c.head()

,Title,Date,Score,2단우산,3단우산,BB/파운데이션/컴팩트류,공병/모델링팩전용도구,귀걸이,기름종이,기타남성화,...,캐쥬얼크로스백,캐쥬얼힙색,크림/밤/오일,팔찌,패션액세서리세트,페이셜클렌저,페이셜팩류,펜던트,학생용가방,헤드웨어
0,같이 살래요,2018-04,28.16,0,0,0,0,928,0,0,...,0,0,0,0,0,0,0,0,32,0
1,같이 살래요,2018-05,28.16,0,0,0,0,928,0,0,...,0,0,0,0,0,0,0,0,32,0
2,같이 살래요,2018-06,28.16,0,0,0,0,928,0,0,...,0,0,0,0,0,0,0,0,32,0
3,같이 살래요,2018-07,28.16,0,0,0,0,928,0,0,...,0,0,0,0,0,0,0,0,32,0
4,같이 살래요,2018-08,28.16,0,0,0,0,928,0,0,...,0,0,0,0,0,0,0,0,32,0


In [0]:
df_c.iloc[:,3:] = df_c.iloc[:,3:].apply(lambda x: x*df_d['Score'])

In [0]:
def normalize(x):
  return (x-np.mean(x))/(np.std(x))

In [0]:
df_4 = df_c[df_c['Date']=='2018-04'].iloc[:,3:].apply(lambda x: sum(x), axis=0).reset_index()
df_4.columns = ['CLAC3_NM','CLAC3_SC']
df_4['CLAC3_SC'] = normalize(df_4['CLAC3_SC'])
df_4['MONTH']='2018-04'

df_5 = df_c[df_c['Date']=='2018-05'].iloc[:,3:].apply(lambda x: sum(x), axis=0).reset_index()
df_5.columns = ['CLAC3_NM','CLAC3_SC']
df_5['CLAC3_SC'] = normalize(df_5['CLAC3_SC'])
df_5['MONTH']='2018-05'

df_6 = df_c[df_c['Date']=='2018-06'].iloc[:,3:].apply(lambda x: sum(x), axis=0).reset_index()
df_6.columns = ['CLAC3_NM','CLAC3_SC']
df_6['CLAC3_SC'] = normalize(df_6['CLAC3_SC'])
df_6['MONTH']='2018-06'

df_7 = df_c[df_c['Date']=='2018-07'].iloc[:,3:].apply(lambda x: sum(x), axis=0).reset_index()
df_7.columns = ['CLAC3_NM','CLAC3_SC']
df_7['CLAC3_SC'] = normalize(df_7['CLAC3_SC'])
df_7['MONTH']='2018-07'

df_8 = df_c[df_c['Date']=='2018-08'].iloc[:,3:].apply(lambda x: sum(x), axis=0).reset_index()
df_8.columns = ['CLAC3_NM','CLAC3_SC']
df_8['CLAC3_SC'] = normalize(df_8['CLAC3_SC'])
df_8['MONTH']='2018-08'

df_9 = df_c[df_c['Date']=='2018-09'].iloc[:,3:].apply(lambda x: sum(x), axis=0).reset_index()
df_9.columns = ['CLAC3_NM','CLAC3_SC']
df_9['CLAC3_SC'] = normalize(df_9['CLAC3_SC'])
df_9['MONTH']='2018-09'


In [0]:
df_45 = pd.concat([df_4,df_5])
df_67 = pd.concat([df_6,df_7])
df_89 = pd.concat([df_8,df_9])
df_4567 = pd.concat([df_45,df_67])
df_t = pd.concat([df_4567,df_89])
df_ct = df_t.groupby(['MONTH','CLAC3_NM'])["CLAC3_SC"].mean().reset_index()

### 합치기

In [0]:
lotte = pd.read_csv("gdrive/My Drive/공모전/Lpoint/데이터셋 만들기/mlotte.csv")

In [0]:
lotte_drama=pd.merge(lotte, df_bt, on=['MONTH','PD_BRA_NM'], how='inner')
lotte_drama=pd.merge(lotte_drama, df_ct, on=['MONTH','CLAC3_NM'], how='inner')

In [262]:
lotte_drama.head()

,MONTH,CLAC1_NM,CLAC2_NM,CLAC3_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,PD_SALES,F_10,F_20,...,Gyeongbu,Gyeongnam,Incheon,Jeju,Jeonbuk,Jeonnam,Seoul,Ulsan,BRA_SC,CLAC3_SC
0,2018-04,남성의류,남성의류상의,남성가디건,갤럭시,239200,1,239200,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.228274,-0.113708
1,2018-04,남성의류,남성의류상의,남성가디건,게스,72507,43,3117801,0.0,1.0,...,1.0,3.0,1.0,0.0,1.0,0.0,16.0,1.0,0.400555,-0.113708
2,2018-04,남성의류,남성의류상의,남성가디건,내셔널지오그래픽,77400,1,77400,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.228274,-0.113708
3,2018-04,남성의류,남성의류상의,남성가디건,라코스테,231750,16,3708000,0.0,3.0,...,1.0,0.0,0.0,0.0,2.0,0.0,6.0,0.0,11.259699,-0.113708
4,2018-04,남성의류,남성의류상의,남성가디건,본,50000,1,50000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.774020,-0.113708


In [0]:
lotte_drama['LABEL'] = lotte_drama[['MONTH','PD_BRA_NM','CLAC3_NM']].apply(lambda x:f'{x[0]}_{x[1]}_{x[2]}', axis=1)
# colnames = ['PD_BRA_NM','CLAC3_NM']
# lotte_drama.drop(colnames, axis=1, inplace=True)
lotte_drama = lotte_drama[['LABEL','CLAC1_NM','CLAC2_NM','PD_BUY_AM', 'PD_BUY_CT', 'PD_SALES', 'BRA_SC', 'CLAC3_SC', 'F_10', 'F_20', 'F_30', 'F_40',
       'F_50', 'F_60', 'F_70', 'F_80', 'M_10', 'M_20', 'M_30', 'M_40', 'M_50',
       'M_60', 'M_70', 'M_80', 'NOT_SIGNED', 'Desktop', 'Mobile', 'Busan',
       'Chungbuk', 'Chungnam', 'Daegu', 'Daejeon', 'Gangwon', 'Gwangju',
       'Gyeonggi', 'Gyeongbu', 'Gyeongnam', 'Incheon', 'Jeju', 'Jeonbuk',
       'Jeonnam', 'Seoul', 'Ulsan']]

In [0]:
lotte_drama.to_csv('gdrive/My Drive/공모전/Lpoint/데이터셋 만들기/mlotte_drama.csv',index=False)

In [265]:
lotte_drama.head()

,LABEL,CLAC1_NM,CLAC2_NM,PD_BUY_AM,PD_BUY_CT,PD_SALES,BRA_SC,CLAC3_SC,F_10,F_20,...,Gwangju,Gyeonggi,Gyeongbu,Gyeongnam,Incheon,Jeju,Jeonbuk,Jeonnam,Seoul,Ulsan
0,2018-04_갤럭시_남성가디건,남성의류,남성의류상의,239200,1,239200,-0.228274,-0.113708,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-04_게스_남성가디건,남성의류,남성의류상의,72507,43,3117801,0.400555,-0.113708,0.0,1.0,...,0.0,7.0,1.0,3.0,1.0,0.0,1.0,0.0,16.0,1.0
2,2018-04_내셔널지오그래픽_남성가디건,남성의류,남성의류상의,77400,1,77400,-0.228274,-0.113708,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-04_라코스테_남성가디건,남성의류,남성의류상의,231750,16,3708000,11.259699,-0.113708,0.0,3.0,...,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,6.0,0.0
4,2018-04_본_남성가디건,남성의류,남성의류상의,50000,1,50000,1.774020,-0.113708,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [266]:
lotte_drama.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16571 entries, 0 to 16570
Data columns (total 43 columns):
LABEL         16571 non-null object
CLAC1_NM      16571 non-null object
CLAC2_NM      16571 non-null object
PD_BUY_AM     16571 non-null int64
PD_BUY_CT     16571 non-null int64
PD_SALES      16571 non-null int64
BRA_SC        16571 non-null float64
CLAC3_SC      16571 non-null float64
F_10          16571 non-null float64
F_20          16571 non-null float64
F_30          16571 non-null float64
F_40          16571 non-null float64
F_50          16571 non-null float64
F_60          16571 non-null float64
F_70          16571 non-null float64
F_80          16571 non-null float64
M_10          16571 non-null float64
M_20          16571 non-null float64
M_30          16571 non-null float64
M_40          16571 non-null float64
M_50          16571 non-null float64
M_60          16571 non-null float64
M_70          16571 non-null float64
M_80          16571 non-null float64
NOT_SIGNED  